In [1]:
%matplotlib inline
!pip install tensorboardcolab

import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import tensorboardcolab

Using TensorFlow backend.


In [2]:
# load cifar10 dataset 
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
# reshape (None, 1) -> (None)
y_train, y_test = [np.reshape(y_train, [-1]), np.reshape(y_test, [-1])]

# normalization 
x_train, x_test = [(x_train - x_train.max()) / (x_train.max() - x_train.min()),
                   (x_test - x_test.max()) / (x_test.max() - x_test.min())]

# N class
n_classes = 10
print('image shape : {}, label shape : {} '.format(x_train.shape, y_train.shape))
print('image shape : {}, label shape : {} '.format(x_test.shape, y_test.shape))
print('train minimun : {}, train_maximum : {} '.format(x_train.min(), x_train.max()))
print('tests minimun : {}, test_maximum : {} '.format(x_test.min(), x_test.max()))

image shape : (50000, 32, 32, 3), label shape : (50000,) 
image shape : (10000, 32, 32, 3), label shape : (10000,) 
train minimun : 0.0, train_maximum : 1.0 
tests minimun : 0.0, test_maximum : 1.0 


In [4]:
temp = x_train
ratio = int(len(x_train) * 0.7)
ratio_end = int(len(x_train) * 1.0)

x_train = temp[0:ratio, :, :, :]
x_validation = temp[ratio:ratio_end , :, :, :]

y_train_label = y_train[0:ratio]
y_validation_label = y_train[ratio:ratio_end ]

print(y_train_label[:10]) # label이 one_hot encoding상태가 아니다.

print(x_train.shape, y_train_label.shape)
print(x_validation.shape, y_validation_label.shape)

[6 9 9 4 1 1 2 7 8 3]
(35000, 32, 32, 3) (35000,)
(15000, 32, 32, 3) (15000,)


## Data Provider

In [0]:
class DataProvider(object):
    def __init__(self, x, y):
        self.epoch_count = 0
        
        self.data = x
        self.label = y
        
        npr.seed(42)
        
        self.indices = self.generate_indices()
        
    def generate_indices(self):
        indices = list(range(len(self.data)))
        npr.shuffle(indices)
        
        return indices
    
    def next_batch(self, batch_size):
        idx = batch_size
        if len(self.indices) < batch_size:
            print("all data consumed, epoch + 1")
            self.epoch_count += 1
            self.indices = self.generate_indices()
    
        target_indices = self.indices[:batch_size]
        del self.indices[:batch_size]
        
        return self.data[target_indices] , self.label[target_indices]

In [0]:
def cifar_generator(data, labels, batch_size=32):
    start_idx = 0
    num_step = len(data) // batch_size
    indexes = np.arange(0, len(data))
    while True:
        if start_idx >= num_step-1:
            np.random.shuffle(indexes)
            start_idx = 0
        else:
            start_idx += 1            
        batch_index = indexes[start_idx*batch_size:
                              (start_idx+1)*batch_size]

        batch_data = data[batch_index]
        batch_label = labels[batch_index]

        yield batch_data, batch_label

## Model B : Build  --> Model A와 동일한 Structure이고, Optimizer만 Momentum으로 변경함

In [18]:
graph = tf.Graph()
with graph.as_default() :
    xs = tf.placeholder(tf.float32, (None, 32, 32, 3), name='xs') # 32*32 = 1024
    ys = tf.placeholder(tf.int32, (None), name='ys')
    lr = tf.placeholder_with_default(0.001, (), name='lr')
    
    with tf.name_scope('convolution_layer_1') :   ## filters is number of channels
        # kernel_init = tf.random.normal(shape=[2, 2, 3, 16], mean=0.0, stddev=0.1, dtype=tf.float32, name='kernel_init') # random normal 
        kernel_init = tf.zeros([2, 2, 3, 16])
        kernel= tf.Variable(kernel_init, name='kernel')               
        bias_init = tf.zeros([16])
        bias = tf.Variable(bias_init, name='bias')
    
        layer = tf.nn.conv2d(xs, kernel, strides=[1,1,1,1], padding='SAME') # convolution and add bias
        layer = layer + bias
        layer = tf.nn.relu(layer)
        
        pool = tf.layers.MaxPooling2D(pool_size=[2,2], strides=[2,2])(layer)

    with tf.name_scope('convolution_layer_2') :   ## filters is number of channels
        kernel_init = tf.zeros([2, 2, 16, 32]) 
        kernel= tf.Variable(kernel_init, name='kernel')               
        bias_init = tf.zeros([32])
        bias = tf.Variable(bias_init, name='bias')
    
        layer = tf.nn.conv2d(pool, kernel, strides=[1,1,1,1], padding='SAME') # convolution and add bias   
        layer = layer + bias
        layer = tf.nn.relu(layer)
           
        pool = tf.layers.MaxPooling2D(pool_size=[2,2], strides=[2,2])(layer)
        
            
    with tf.name_scope('convolution_layer_3') :   ## filters is number of channels
        kernel_init = tf.zeros([3, 3, 32, 64])
        kernel= tf.Variable(kernel_init, name='kernel')               
        bias_init = tf.zeros([64])
        bias = tf.Variable(bias_init, name='bias')    
        layer = tf.nn.conv2d(pool, kernel, strides=[1,1,1,1], padding='SAME') # convolution and add bias       
        layer = layer + bias
        layer = tf.nn.relu(layer)

        kernel_init2 = tf.zeros(shape=[3, 3, 64, 64]) 
        kernel2= tf.Variable(kernel_init2, name='kernel')               
        bias_init2 = tf.zeros([64])
        bias2 = tf.Variable(bias_init2, name='bias')    
        layer = tf.nn.conv2d(layer, kernel2, strides=[1,1,1,1], padding='SAME') # convolution and add bias   
        layer = layer + bias2
        layer = tf.nn.relu(layer)
    
        pool = tf.layers.MaxPooling2D(pool_size=[2,2], strides=[2,2])(layer)        
        
    # receptive filed = 27, images size = 32, output imagesize = (4, 4)
    
    with tf.name_scope('Reconstruct') : 
        flatten = tf.layers.flatten(pool)
        layer = tf.layers.Dense(1024, activation=tf.nn.relu)(flatten) # 32*32
        layer = tf.layers.Dense(1024, activation=tf.nn.relu)(layer)
        y_pred = tf.layers.Dense(10, activation=None, name='y_pred')(layer)
        
    with tf.name_scope('Loss') :
        loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=ys, logits=y_pred))
        print(loss.shape)
        
    with tf.name_scope('metric') :
        rmse = tf.sqrt(loss)
    
    with tf.name_scope('accuracy') :
        pred = tf.cast(tf.arg_max(y_pred, 1), tf.int32)
        correct = tf.equal(pred, ys)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        
        # add tensor to tensorboard
        acc_tb = tf.summary.scalar(name='accuracy', tensor=accuracy)

    with tf.name_scope('train') :
        # train_op = tf.train.AdamOptimizer(lr).minimize(loss)
        train_op = tf.train.MomentumOptimizer(lr, momentum=0.9, use_nesterov=False).minimize(loss)
        

()


## Model B : Train

### Load trained weight and bias

In [0]:
from tensorflow.python.training import checkpoint_utils as cp

# 변수 불러오기
var_names = cp.list_variables('./model/vgg_net')   

# 값 불러오기
con_layer1_kernel  = cp.load_variable('./model/vgg_net' , 'convolution_layer_1/kernel')   
con_layer1_bias    = cp.load_variable('./model/vgg_net' , 'convolution_layer_1/bias')

con_layer2_kernel  = cp.load_variable('./model/vgg_net' , 'convolution_layer_2/kernel')   
con_layer2_bias    = cp.load_variable('./model/vgg_net' , 'convolution_layer_2/bias')

con_layer3_kernel1 = cp.load_variable('./model/vgg_net' , 'convolution_layer_3/kernel')   
con_layer3_bias1   = cp.load_variable('./model/vgg_net' , 'convolution_layer_3/bias')
con_layer3_kernel2 = cp.load_variable('./model/vgg_net' , 'convolution_layer_3/kernel_1')   
con_layer3_bias2   = cp.load_variable('./model/vgg_net' , 'convolution_layer_3/bias_1')

## Model B : Train

In [19]:
with graph.as_default() :
    
    log_dir = "./log/vgg_net_model_b"   
    lode_dir = "./model/vgg_net"
    save_dir = "./model/vgg_net_model_b"    
    
    # 텐서보드에 연결합니다.
    tbc = tensorboardcolab.TensorBoardColab(graph_path = log_dir)
    train_writer = tf.summary.FileWriter(logdir = log_dir)
    train_writer.add_graph(tf.get_default_graph())
    merged_all = tf.summary.merge_all()    
    
    sess = tf.Session()
    # weight를 불러왔기 때문에 초기화를 하면 안됨!
    sess.run(tf.global_variables_initializer()) 
    
    # Step2 : 기록
    saver = tf.train.Saver()
    
    # loading parameters
    saver.restore(sess, save_path = lode_dir)
    # convolution_layer_1/kernel/Momentum <-- 얘를 초기화 하지 못함
    
    # Training
    batch_size = 50
    n_epoch = 30
    n_step = int(len(x_train) // batch_size)  # //은 몫이다.
    learing_rate = 0.0001
    
    # instance 생성
    train_generator = cifar_generator(x_train, y_train_label, batch_size)
    
    train_loss = []
    valid_loss = []
    valid_acc = []
    cnt = 0
    minimum_loss = 1.1
    for i in tqdm(range(n_epoch)) :
        for step in range(n_step) :
            batch_xs, batch_ys = next(train_generator)
            _, train_loss_, tbs_train_ = sess.run([train_op, rmse, merged_all], feed_dict = { xs: batch_xs, 
                                                                                              ys: batch_ys, 
                                                                                              lr: learing_rate})
            train_writer.add_summary(tbs_train_, global_step=cnt) # 흠 되야 하는데 안된다.
            cnt += 1
            train_loss.append(train_loss_)
            
            # check validation set
            if step % 100 == 0 :
                loss_, acc_ = sess.run([rmse, accuracy], feed_dict = { xs: x_validation, 
                                                                       ys: y_validation_label})
                valid_loss.append(loss_)
                valid_acc.append(acc_)
                
                # Save the model
                if loss_ < minimum_loss :
                    print("log current model!")
                    minimum_loss = loss_
                    saver.save(sess, save_path = save_dir)
        print("loss = {:.4f}, acc = {:.2f}%".format(loss_, acc_*100))
    print("loss = {:.4f}, acc = {:.2f}%".format(loss_, acc_*100))
    
    train_writer.flush() # file을 disk에 쓴다

Wait for 8 seconds...
TensorBoard link:
https://479415ea.ngrok.io


NotFoundError: ignored